In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, udf
from pyspark.sql.types import StringType, BooleanType, IntegerType, FloatType
import random
import hashlib

In [ ]:
# -------------------------------------
# Step 1: Initialize Spark
# -------------------------------------
spark = SparkSession.builder \
    .appName("Reddit Comments Scrambling") \
    .getOrCreate()


In [ ]:
# -------------------------------------
# Step 2: Data Ingestion (Bronze Layer)
# -------------------------------------

# TODO: Needs to have files on hdfs
raw_data_path = "hdfs://your_hdfs_path/data/subset/100000-reddit-covid-comments.csv"  # Modify this path as needed
raw_df = spark.read.option("header", "true").csv(raw_data_path)

# Display raw data sample
print("Raw Data Sample:")
raw_df.show(5)

# Save raw data into the Bronze layer
raw_data_bronze_path = "data/bronze_layer/"
raw_df.write.mode("overwrite").parquet(raw_data_bronze_path)

In [ ]:
# -------------------------------------
# Step 3: Data Scrambling
# -------------------------------------
def scramble_row(type_field, id_field, subreddit_id, subreddit_name, subreddit_nsfw, created_utc, permalink, body, sentiment, score):
    # Replace '|' in body to avoid conflicts with new delimiter
    body = body.replace('|', '__PIPE__')  
    type_field = (type_field == "comment")  # Convert to boolean
    subreddit_nsfw = "nsfw" if subreddit_nsfw == "true" else "not_nsfw"  # Convert to string

    # Create a scrambled line with "|" as delimiter
    row_str = f"{type_field}|{id_field}|{subreddit_id}|{subreddit_name}|{subreddit_nsfw}|{created_utc}|{sentiment}|{permalink}|{body}|{score}"
    
    # Randomly break the row into two parts
    if random.random() > 0.7:  # 30% chance to split a row into two parts
        split_point = len(row_str) // 2
        row_str = row_str[:split_point] + '\n' + row_str[split_point:]

    return row_str

# Register the scramble function as a UDF
scramble_udf = udf(scramble_row, StringType())

# Apply the scramble function
scrambled_df = raw_df.select(scramble_udf(
    col("type"),
    col("id"),
    col("subreddit.id"),
    col("subreddit.name"),
    col("subreddit.nsfw"),
    col("created_utc"),
    col("permalink"),
    col("body"),
    col("sentiment"),
    col("score")
).alias("scrambled_data"))


In [ ]:
# -------------------------------------
# Step 4: Data Cleaning (Silver Layer)
# -------------------------------------
def clean_data(scrambled_data):
    lines = scrambled_data.split("\n")
    
    # Step 1: Rejoin broken lines
    cleaned_lines = []
    current_line = ""

    for line in lines:
        if line.strip():  # Ignore empty lines
            current_line += line.strip() + " "  # Append with a space to avoid issues
            # Check if we have a complete line
            if current_line.count('|') == 9:  # Exactly 10 fields
                cleaned_lines.append(current_line.strip())  # Add completed line
                current_line = ""  # Reset for the next line

    # Handle any remaining line that wasn't appended
    if current_line:
        cleaned_lines.append(current_line.strip())

    # Step 2: Parse each cleaned line back into columns
    parsed_data = []

    for line in cleaned_lines:
        fields = line.split('|')

        if len(fields) < 10:
            continue  # Skip bad rows
        
        # Extract fields
        type_field = "comment" if fields[0] == "True" else "post"  # Convert back to string
        id_field = fields[1]
        subreddit_id = fields[2]
        subreddit_name = fields[3]
        subreddit_nsfw = "nsfw" if fields[4] == "True" else "not_nsfw"  # Convert to string
        created_utc = fields[5]
        permalink = fields[6]
        body = fields[7].replace('__PIPE__', '|')  # Restore original pipe symbol
        score = fields[8]

        # Create the row dictionary
        row_dict = {
            'type': type_field,
            'id': id_field,
            'subreddit.id': subreddit_id,
            'subreddit.name': subreddit_name,
            'subreddit.nsfw': subreddit_nsfw,
            'created_utc': created_utc,
            'permalink': permalink,
            'body': body,
            'score': score
        }
        parsed_data.append(row_dict)

    return parsed_data

# TODO: Is this really using spark and hdfs???

# Register the clean function as a UDF
clean_udf = udf(lambda scrambled: clean_data(scrambled), StringType())

# Apply the clean function
cleaned_df = scrambled_df.select(clean_udf(col("scrambled_data")).alias("cleaned_data"))

In [ ]:
# -------------------------------------
# Step 5: Flatten the cleaned DataFrame
# -------------------------------------
cleaned_df_flat = cleaned_df.selectExpr("explode(cleaned_data) as data").select("data.*")

# Display cleaned data sample
print("Cleaned Data Sample:")
cleaned_df_flat.show(5)

# Save cleaned data to the Silver layer
cleaned_data_silver_path = "data/silver_layer/"
cleaned_df_flat.write.mode("overwrite").parquet(cleaned_data_silver_path)

In [ ]:
# -------------------------------------
# Step 6: Data Serving (Gold Layer)
# -------------------------------------
# Load the cleaned data from the Silver layer
cleaned_df = spark.read.parquet(cleaned_data_silver_path)

# For example, let's find the average sentiment by subreddit
cleaned_df.createOrReplaceTempView("reddit_comments")

# TODO: This is probably too simple, as it just counts the score (thumps up/down)

# Perform sentiment analysis using SQL
sentiment_analysis_df = spark.sql("""
    SELECT subreddit_name, AVG(score) as avg_score, COUNT(*) as comment_count
    FROM reddit_comments
    GROUP BY subreddit_name
    HAVING COUNT(*) > 50  -- Subreddits with more than 50 comments
    ORDER BY avg_score DESC
""")

# Display sentiment analysis results
print("Sentiment Analysis Results:")
sentiment_analysis_df.show(10)

# Save the final data to the Gold layer
final_data_gold_path = "data/gold_layer/"
sentiment_analysis_df.write.mode("overwrite").parquet(final_data_gold_path)